<a href="https://colab.research.google.com/github/DarkPovoGang/DeepRL/blob/main/DeepRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-lxle_e39
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-lxle_e39
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369370 sha256=6ecf7dbd79d16cb2f98bed5f3f245aa8680f707345dd4f9ebbdc5f195f646a9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-on2nq_l1/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [3]:
import pandas as pd
import os
import gdown
import tarfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
class RefCOCOg(Dataset):

  FILE_ID = '1wyyksgdLwnRMC9pQ-vjJnNUn47nWhyMD'
  ARCHIVE_NAME = 'refcocog.tar.gz'
  NAME = 'refcocog'
  ANNOTATIONS = 'annotations/refs(umd).p'
  IMAGES = 'images'
  IMAGE_NAME = 'COCO_train2014_{}.jpg'


  def __init__(self, data_dir, split, transform=None):
      self.data_dir = data_dir
      self._check_dataset()
      self.split = split
      self._filter_annotation(os.path.join(self.data_dir, self.NAME, self.ANNOTATIONS))
      self.transform = transform

  def _check_dataset(self):
      if not os.path.exists(os.path.join(self.data_dir, self.ARCHIVE_NAME)):
          if not os.path.exists(self.data_dir):
              os.mkdir(self.data_dir)
          print('Downloading dataset...')
          gdown.download(id=self.FILE_ID)
      if not os.path.exists(os.path.join(self.data_dir, self.NAME)):
          print('Extracting dataset...')
          with tarfile.open(os.path.join(self.data_dir, self.ARCHIVE_NAME), 'r:gz') as tar:
              tar.extractall(path=self.data_dir)
      else:
          print('Dataset already extracted')

  def __len__(self):
      return len(self.annotation)

  def __getitem__(self, idx, tensor=True):
      # get line by index
      raw = self.annotation.iloc[idx]
      # get image
      image = self._get_image(raw, tensor)
      # get sentences
      sentences = self._get_sentences(raw)
      return image,sentences

  def _get_image(self, raw):
      # get image_id
      image_id = raw['image_id']
      # pad image_id to 12 digits
      image_id = str(image_id).zfill(12)
      # convert image to tensor
      image = Image.open(os.path.join(self.data_dir, self.NAME, self.IMAGES, self.IMAGE_NAME.format(image_id)))
      return image

  def _get_sentences(self, raw):
      # get sentences
      sentences = raw['sentences']
      # get raw sentences
      sentences = [sentence['raw'] for sentence in sentences]
      return sentences
  def _filter_annotation(self, path):
      # self.annotation = pd.read_pickle(path)
      # #self.annotation = pd.DataFrame([x for x in (self.annotation) if x['split'] == 'val'])
      # self.annotation = [x for x in (self.annotation) if x['split'] == 'val']
      # self.ANNOTATION_SIZE = len(self.annotation)
      self.annotation = pd.read_pickle(path)
      self.annotation = pd.DataFrame(self.annotation)
      self.annotation = self.annotation[self.annotation['split'] == self.split]

In [5]:
import numpy as np
import torch
from pkg_resources import packaging
import clip
print("Torch version:", torch.__version__)
model, preprocess = clip.load("RN50")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

preprocess

Torch version: 2.0.1+cu118


100%|███████████████████████████████████████| 244M/244M [00:03<00:00, 84.6MiB/s]


RuntimeError: ignored